In [58]:
load('~/HLI/hla/test.rda')

In [60]:
allele.names <- colnames(test)
allele.genes <- unique(sub('\\*.+', '', allele.names))
alleles <- 1:ncol(test)
reads <- 1:nrow(test)
na <- length(alleles)
nr <- length(reads)

In [61]:
library(lpSolve)

In [62]:
f.obj <- c(rep(0, na), ifelse(is.specific, 1, 0.1))
# constraints for hit incidence matrix
f.con <- do.call(rbind, lapply(1:nr, function(i){
    con <- c(test[i,], rep(0, nr))
    con[na + i] <- -1
    con
}))
f.dir <- rep('>=', nr)
f.rhs <- rep(0, nr)
# constraints for 1 or 2 alleles per gene

f.con2 <- do.call(rbind, lapply(allele.genes, function(gene){
    con <- c(rep(0, na), rep(0, nr))
    con[grep(sprintf("^%s", gene), allele.names)] <- 1
    rbind(con, con)
}))
f.dir2 <- rep(c('>=', '<='), length(allele.genes))
f.rhs2 <- rep(c( 1,    2  ), length(allele.genes))

# final constraints
f.con <- rbind(f.con, f.con2)
f.dir <- c(f.dir, f.dir2)
f.rhs <- c(f.rhs, f.rhs2)

In [63]:
system.time(lps <- lp('max', f.obj, f.con, f.dir, f.rhs, all.bin = T))

   user  system elapsed 
  2.827   0.070   2.899 

In [64]:
lps

Success: the objective function is 508 

In [65]:
solution <- lps$solution[alleles]
names(solution) <- allele.names
solution <- solution[order(-solution)]
solution <- solution[solution > 0]
solution

A*02:01  A*03:01  B*13:02  B*51:01  C*01:11 C*06:109 
       1        1        1        1        1        1

In [66]:
nrow(f.con)

[1] 575

In [67]:
ncol(f.con)

[1] 7015